In [7]:
import requests
from requests import post, get
import base64
import urllib
import os
import json
import pandas as pd
from dotenv import load_dotenv
import plotly.express as px
import pickle
import schedule
import time

In [9]:
# Spotify Developer Dashboard details
load_dotenv()
client_id = os.getenv("CLIENT_ID")
client_secret = os.getenv("CLIENT_SECRET")
print(type(client_secret))

<class 'str'>


In [10]:
# Get Spotify token   
def get_token():
    auth_string = client_id + ':' + client_secret
    auth_bytes = auth_string.encode("utf-8")
    # Spotify requests to encode the client ID and client secret using base64
    auth_base64 = str(base64.b64encode(auth_bytes), "utf-8")

    url = "https://accounts.spotify.com/api/token"
    headers = {
        "Authorization": "Basic " + auth_base64,
        "Content-Type": "application/x-www-form-urlencoded"
    }
    data = {"grant_type": "client_credentials"}
    # Make the request
    result = post(url,headers= headers, data= data)
    # Convert the result (json) into a python dictionary
    json_result = json.loads(result.content) 
    token = json_result["access_token"]
    return token

In [11]:
token = get_token()

In [12]:
# Function to construct the header to send a request
def get_auth_header(token):
    return{"Authorization": "Bearer " + token}


In [35]:
# Get new releases
def get_new_releases(df, country):
    # Find the country code for the given country name
    row = df[df['country'] == country]
    country_code = row.iloc[0]['country_code']
    url= f"https://api.spotify.com/v1/browse/new-releases?country={country_code}&limit=1"
    headers = get_auth_header(token)
    result = get(url, headers=headers)
    json_result = json.loads(result.content)['albums']['items'][0]['name']
    return json_result

In [36]:
# Get artist new releases
def get_artist_new_releases(df,country):
    # Find the country code for the given country name
    row = df[df['country'] == country]
    country_code = row.iloc[0]['country_code']
    url= f"https://api.spotify.com/v1/browse/new-releases?country={country_code}&limit=1"
    headers = get_auth_header(token)
    result = get(url, headers=headers)
    json_result = json.loads(result.content)['albums']['items'][0]['artists'][0]['name']
    return json_result

In [37]:
def get_new_releases_id(df, country):
    # Find the country code for the given country name
    row = df[df['country'] == country]
    country_code = row.iloc[0]['country_code']
    url= f"https://api.spotify.com/v1/browse/new-releases?country={country_code}&limit=1"
    headers = get_auth_header(token)
    result = get(url, headers=headers)
    # Get the album ID
    json_result = json.loads(result.content)['albums']['items'][0]['id']
    return json_result

In [47]:
# Get album image
def get_new_releases_pic(df,country):
    # Find the country code for the given country name
    row = df[df['country'] == country]
    country_code = row.iloc[0]['country_code']
    url= f"https://api.spotify.com/v1/browse/new-releases?country={country_code}&limit=1"
    headers = get_auth_header(token)
    result = get(url, headers=headers)
    # Get the image url
    json_result = json.loads(result.content)['albums']['items'][0]['images'][1]['url']
    return json_result

In [48]:
# Function to update new releases in the dataframe
def update_new_releases(pickle_file_path):
    # Load the new_releases
    with open(pickle_file_path, 'rb') as file:
        df_countries = pickle.load(file)

    #Modify for Namibia: it detects NaN instead of NA country code
    df_countries.iloc[115,4]='NA' 
        
    # Get the new releases per country
    df_countries['new_releases'] = [get_new_releases(df_countries, df_countries.loc[index, 'country']) for index in range(len(df_countries))]

    # Get the new release artist
    df_countries['artist'] = [get_artist_new_releases(df_countries, df_countries.loc[index, 'country']) for index in range(len(df_countries))]

    # Get the album ID for the list of tracks
    df_countries['album_id'] = [get_new_releases_id(df_countries, df_countries.loc[index, 'country']) for index in range(len(df_countries))]

    # Get the cover image url
    df_countries['image_url'] = [get_new_releases_pic(df_countries, df_countries.loc[index, 'country']) for index in range(len(df_countries))]

     # Save the updated dataframe to a temporary pickle file
    temp_pickle_file_path = os.path.splitext(pickle_file_path)[0] + '_temp.pkl'
    with open(temp_pickle_file_path, 'wb') as file:
        pickle.dump(df_countries, file)

    # Replace the original pickle file with the updated one
    os.replace(temp_pickle_file_path, pickle_file_path)
    print("New releases updated successfully!")

In [49]:
pickle_file_path= r"C:\Users\sofia\OneDrive\Documentos\GitHub\bigdataproject\app\data\new_releases.pkl"
update_new_releases(pickle_file_path)

New releases updated successfully!


In [50]:
df_countries = pd.read_pickle(r"C:\Users\sofia\OneDrive\Documentos\GitHub\bigdataproject\app\data\new_releases.pkl")

In [51]:
print(df_countries[df_countries['country']=="Argentina"])
country= "Argentina"

     country       capital    cap_lat    cap_lon country_code      continent  \
7  Argentina  Buenos Aires -34.583333 -58.666667           AR  South America   

                          new_releases   artist                album_id  \
7  MAÑANA SERÁ BONITO (BICHOTA SEASON)  KAROL G  1f2q2JQ3GFwIrWch2JLC0u   

                                           image_url  
7  https://i.scdn.co/image/ab67616d00001e024a8c1b...  


In [52]:
# With the ID, get the tracks of that album
def tracks_from_album(country):
    # Find the country code for the given country name
    row = df_countries[df_countries['country'] == country]
    album_id = row.iloc[0]['album_id']
    url= f"https://api.spotify.com/v1/albums/{album_id}/tracks?limit=15"
    headers = get_auth_header(token)
    result = get(url, headers=headers)
    json_result = json.loads(result.content)
    # List the tracks
    tracks_data= []
    for item in json_result['items']:
        name=item['name']
        tracks_data.append({'Tracks': name})
    tracks_df= pd.DataFrame(tracks_data)
    return tracks_df


In [53]:
print(tracks_from_album(country))
#selected_album= df_countries.loc[df_countries['country']== country,'new_releases'].values[0]
#selected_artist= df_countries.loc[df_countries['country']== country,'artist'].values[0]
#image_url= get_new_releases_pic(country)


                          Tracks
0                       BICHOTAG
1                       OKI DOKI
2              MI EX TENÍA RAZÓN
3                            S91
4                          QLONA
5  UNA NOCHE EN MEDELLÍN - REMIX
6                ME TENGO QUE IR
7                GATITA GANGSTER
8                          DISPO
9               PROVENZA - REMIX


# Map

In [15]:
df.iloc[115,4]

nan

In [16]:
df.iloc[115,4]='NA'

In [18]:
df['new_releases']= [get_new_releases(df.loc[index,'country']) for index in range(len(df))]

KeyboardInterrupt: 

In [43]:
df['artist']= [get_artist_new_releases(df.loc[index,'country']) for index in range(len(df))]

In [44]:
df.head()

,country,capital,cap_lat,cap_lon,country_code,continent,new_releases,artist
0,Palestine,Jerusalem,31.766667,35.233333,PS,Asia,TUYA,ROSALÍA
1,Nauru,Yaren,-0.547700,166.920867,NR,Australia,BLONDE16,Miliyah
2,Albania,Tirana,41.316667,19.816667,AL,Europe,Ku Je Ti (feat. Ricky Rich & Dafina Zeqiri),DYSTINCT
3,Algeria,Algiers,36.750000,3.050000,DZ,Africa,YA LE3DOUWA,Zouhair Bahaoui
4,Andorra,Andorra la Vella,42.500000,1.516667,AD,Europe,Drapeau Blanc,Benab


In [29]:
fig = px.scatter_geo(df, lat='cap_lat', lon= 'cap_lon',
                     hover_name='new_releases', color='country',
                     hover_data={'cap_lat': False, # Don't show latitude and longitud
                                 'cap_lon': False,
                                 'artist': True,   # Show artist and country
                                 'country': True}
                     ) 

# Update the layout with the desired colors
'''fig.update_layout(
    geo=dict(
        bgcolor='black',  # Set the background color to black
        showland=True,
        landcolor='white',  # Set the color of land areas to white
        showocean=True,
        oceancolor='black',  # Set the color of ocean areas to black
        showlakes=True,
        lakecolor='white'  # Set the color of lakes to white
    ),
    font=dict(
        color='white'  # Set the text color to white
    )
)'''
fig.show()

In [27]:
#test = df.groupby('continent')['continent'].count()
fig2 = px.scatter_mapbox(df, lat='cap_lat', lon='cap_lon', title="Number of countries")
pyo.iplot(fig2)

# Updates

In [19]:
# Read country data
df_countries= pd.read_excel(r'C:\Users\sofia\OneDrive\Documentos\GitHub\bigdataproject\app\data\country-available-final.xlsx')
df_countries.iloc[115,4]='NA' #Modify for Namibia: it detects NaN instead of NA country code


In [20]:
# Function to update new releases in the dataframe
def update_new_releases(pickle_file_path):
    # Load the new_releases
    with open(pickle_file_path, 'rb') as file:
        df_countries = pickle.load(file)
    
    #Modify for Namibia: it detects NaN instead of NA country code
    df_countries.iloc[115,4]='NA' 
        
    # Get the new releases per country
    df_countries['new_releases'] = [get_new_releases(df_countries.loc[index, 'country']) for index in range(len(df_countries))]

    # Get the new release artist
    df_countries['artist'] = [get_artist_new_releases(df_countries.loc[index, 'country']) for index in range(len(df_countries))]

     # Save the updated dataframe to a temporary pickle file
    temp_pickle_file_path = os.path.splitext(pickle_file_path)[0] + '_temp.pkl'
    with open(temp_pickle_file_path, 'wb') as file:
        pickle.dump(df_countries, file)

    # Replace the original pickle file with the updated one
    os.replace(temp_pickle_file_path, pickle_file_path)
    print("New releases updated successfully!")


In [21]:
pickle_file_path= r"C:\Users\sofia\OneDrive\Documentos\GitHub\bigdataproject\app\data\new_releases.pkl"
update_new_releases(pickle_file_path)

New releases updated successfully!


In [22]:
file= pd.read_pickle(r'C:\Users\sofia\OneDrive\Documentos\GitHub\bigdataproject\app\data\new_releases.pkl')
file.head()

,country,capital,cap_lat,cap_lon,country_code,continent,new_releases,artist
0,Palestine,Jerusalem,31.766667,35.233333,PS,Asia,TUYA,ROSALÍA
1,Nauru,Yaren,-0.547700,166.920867,NR,Australia,BLONDE16,Miliyah
2,Albania,Tirana,41.316667,19.816667,AL,Europe,Ku Je Ti (feat. Ricky Rich & Dafina Zeqiri),DYSTINCT
3,Algeria,Algiers,36.750000,3.050000,DZ,Africa,Zarzour,Lartiste
4,Andorra,Andorra la Vella,42.500000,1.516667,AD,Europe,Alchemy,Disclosure


In [23]:
file[file['country']=='Argentina']

,country,capital,cap_lat,cap_lon,country_code,continent,new_releases,artist
7,Argentina,Buenos Aires,-34.583333,-58.666667,AR,South America,Antes de Ameri,Duki


In [47]:

pickle_file_path= r"C:\Users\sofia\OneDrive\Documentos\GitHub\bigdataproject\app\data\new_releases.pkl"
schedule.every(3).days.at("00:00").do(update_new_releases, pickle_file_path)

while True: # This starts an infinite loop.
    schedule.run_pending() # This line checks if there are any scheduled tasks that are due to run and executes them. If there are no pending tasks, it does nothing.
    time.sleep(1) #After checking for pending tasks, the program pauses for 1 second using the time.sleep() function. This ensures that the loop doesn't consume excessive resources and provides a delay between iterations.

KeyboardInterrupt: 

In [17]:
df_countries.loc[5:183]

,country,capital,cap_lat,cap_lon,country_code,continent,new_releases,artist
5,Angola,Luanda,-8.833333,13.216667,AO,Africa,Harry's House,Harry Styles
6,Antigua and Barbuda,Saint John's,17.116667,-61.850000,AG,North America,Harry's House,Harry Styles
7,Argentina,Buenos Aires,-34.583333,-58.666667,AR,South America,El Final de Las Cosas,Barbi Recanati
8,Armenia,Yerevan,40.166667,44.500000,AM,Europe,Harry's House,Harry Styles
9,Australia,Canberra,-35.266667,149.133333,AU,Australia,The Winding Way,The Teskey Brothers
...,...,...,...,...,...,...,...,...
178,Vietnam,Hanoi,21.033333,105.850000,VN,Asia,Promise,Laufey
179,Zambia,Lusaka,-15.416667,28.283333,ZM,Africa,Harry's House,Harry Styles
180,Zimbabwe,Harare,-17.816667,31.033333,ZW,Africa,Harry's House,Harry Styles
181,Hong Kong,NaN,0.000000,0.000000,HK,Asia,Promise,Laufey
